In [1]:
# FUNCTION TO CALCULATE WEIGHTED T SCORE FOR EACH PARTICLE IN THE POPULATION

import random
from math import *
import copy
import numpy

def weighted_t_score(Normal_Values,Tumor_Values,N_Value,T_Value,normal_count,tumor_count,GENE_IN_PARTICLE):
    
    DEG_OF_RELEVANCE=[random.uniform(0,1) for i in range(GENE_IN_PARTICLE)]
    #print DEG_OF_RELEVANCE
    for i in range(GENE_IN_PARTICLE):
        Normal_Values[i]=(Normal_Values[i]*DEG_OF_RELEVANCE[i])
    for i in range(GENE_IN_PARTICLE):
        Tumor_Values[i]=(Tumor_Values[i]*DEG_OF_RELEVANCE[i])
    
    FINAL_VALUE1=0.0
    FINAL_WEIGHT=0.0
    FINAL_VALUE2=0.0
    
    for i in range(GENE_IN_PARTICLE):
        FINAL_VALUE1+=Normal_Values[i]
        FINAL_WEIGHT+=DEG_OF_RELEVANCE[i]
    
                               
    for i in range(GENE_IN_PARTICLE):
        FINAL_VALUE2+=Tumor_Values[i]
           
    Mean1=FINAL_VALUE1/FINAL_WEIGHT # weighted mean of class 1
    Mean2=FINAL_VALUE2/FINAL_WEIGHT # weighted mean of class 2
   
    for i in range(GENE_IN_PARTICLE):
        N_Value[i]=(abs(N_Value[i]-Mean1)**2)*DEG_OF_RELEVANCE[i]
        
        
    for i in range(GENE_IN_PARTICLE):
        T_Value[i]=(abs(T_Value[i]-Mean2)**2)*DEG_OF_RELEVANCE[i]
        
    FINAL_STD1=0.0
    FINAL_STD2=0.0
    for i in range(GENE_IN_PARTICLE):
        FINAL_STD1+=N_Value[i]
                               
    for i in range(GENE_IN_PARTICLE):
        FINAL_STD2+=T_Value[i]
        
    
    StdDev1=(((normal_count/(normal_count-1))*(FINAL_STD1/FINAL_WEIGHT))**0.5) # weighted standard deviation of class 1
    StdDev2=(((tumor_count/(tumor_count-1))*(FINAL_STD2/FINAL_WEIGHT))**0.5) # weighted standard deviation of class 2
    
    WEIGHTED_T_SCORE=(abs(Mean1-Mean2))/(((StdDev1/normal_count)+(StdDev2/tumor_count))**0.5)
    return(WEIGHTED_T_SCORE)

def Weighted_T_Score(POPULATION_SIZE):
    
    no_of_pathways=0
    NO_OF_GENES = []
    NumberOfGenes=[]
    arr1=list()
    GENES_IN_PARTICLE=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/pathway/child_path(copy).txt") as br:
        for line in br:
            if(line.startswith("KEGG")):
                no_of_pathways+=1
       
    GENES=[[] for j in range(no_of_pathways)]
    i=0
                
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/pathway/child_path(copy).txt") as br:
        for line in br:
            if(line.startswith("KEGG")):
                arr1=line.split('\t')
                GENES[i]=arr1[5].split(", ")
                NO_OF_GENES.append(len(GENES[i]))
                NumberOfGenes.append(len(GENES[i]))
                GENES_IN_PARTICLE+=len(GENES[i])
                i+=1
   

    NORMAL_DATA=[]
    TUMOR_DATA=[]
    n_count=0
    t_count=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as fp:
        for line in fp:
            if(line.startswith("normal")):
                n_count+=1
            elif(line.startswith("tumor")):
                t_count+=1
       
    COUNT=n_count+t_count
    ALL_GENES=[]
    Total=0
    VALUES=[]
    Normal_Values=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    N_Value=[float(0.0) for i  in range(GENES_IN_PARTICLE)]
    Tumor_Values=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    T_Value=[float(0.0)for i in range(GENES_IN_PARTICLE)]
    g=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as sn:
        for line in sn:
            if(line.startswith("class")):
                ALL_GENES=line.split("\t")
    #print GENES           
    for i in range(no_of_pathways):
        for j in range(NumberOfGenes[i]):
            for k in range(len(ALL_GENES)):
                if(GENES[i][j].lower()==ALL_GENES[k]):
                    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as am:
                        for line in am:
                            if(line.startswith("normal")):
                                VALUES=line.split("\t")
                                Normal_Values[g]+=float(VALUES[k])
                                N_Value[g]+=float(VALUES[k])
                            elif(line.startswith("tumor")):
                                VALUES=line.split("\t")
                                Tumor_Values[g]+=float(VALUES[k])
                                T_Value[g]+=float(VALUES[k])
            g+=1
            
    T_SCORE=[[] for i in range(POPULATION_SIZE)]
    for q in range(POPULATION_SIZE):
        #print ("Weighted T Score For Particle " +str(q+1)+" :")
        Norm_Values=copy.copy(Normal_Values)
        Tum_Values=copy.copy(Tumor_Values)
        N_Values=copy.copy(N_Value)
        T_Values=copy.copy(T_Value)
        
        T_SCORE[q]=weighted_t_score(Normal_Values,Tumor_Values,N_Value,T_Value,n_count,t_count,GENES_IN_PARTICLE)
        
        Normal_Values=copy.copy(Norm_Values)
        Tumor_Values=copy.copy(Tum_Values)
        N_Value=copy.copy(N_Values)
        T_Value=copy.copy(T_Values)
        
    return(T_SCORE)
        
    
        
    